# BP1 - Interpreting random forest using decision rules

-------

In [3]:
import sys
sys.prefix

'/Users/edvin/anaconda3'

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn import preprocessing

ImportError: dlopen(/Users/edvin/anaconda3/lib/python3.7/site-packages/scipy/special/_ufuncs.cpython-37m-darwin.so, 2): Library not loaded: @rpath/libgfortran.3.dylib
  Referenced from: /Users/edvin/anaconda3/lib/python3.7/site-packages/scipy/special/_ufuncs.cpython-37m-darwin.so
  Reason: image not found

In [ ]:
import operator

def get_truth(inp, relate, cut):
    ops = {'>': operator.gt,
           '<': operator.lt,
           '>=': operator.ge,
           '<=': operator.le,
           '=': operator.eq}
    return ops[relate](inp, cut)

In [ ]:
from sklearn.exceptions import UndefinedMetricWarning
import warnings

warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

In [ ]:
data = pd.read_csv('datasets/iris-data.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data['class'].value_counts()

In [ ]:
data.loc[data['class'] == 'Iris-setossa', 'class'] = 'Iris-setosa'
data.loc[data['class'] == 'versicolor', 'class'] = 'Iris-versicolor'

In [ ]:
data.head()

In [ ]:
data['class'].unique()

In [ ]:
sns.kdeplot(data.loc[data['class'] == 'Iris-setosa', 'petal_width_cm'], shade=True, label='setosa')
sns.kdeplot(data.loc[data['class'] == 'Iris-versicolor', 'petal_width_cm'], shade=True, label='versicolor')
sns.kdeplot(data.loc[data['class'] == 'Iris-virginica', 'petal_width_cm'], shade=True, label='virginica')

In [ ]:
data[data.isnull().any(axis=1)]

In [ ]:
setosa_petal_width_mean = data.loc[data['class'] == 'Iris-setosa', 'petal_width_cm'].mean()

In [ ]:
data.loc[(data['class'] == 'Iris-setosa') & data['petal_width_cm'].isnull(), 'petal_width_cm'] = setosa_petal_width_mean

In [ ]:
data.plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
plt.show()

In [ ]:
sns.pairplot(data, hue="class")

In [ ]:
data.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.heatmap(data.corr(), ax=ax, annot=True, fmt=".3f")

-----
-----
-----
-----

In [ ]:
from sklearn import model_selection

In [ ]:
array = data.values
X = array[:,0:4]
Y = array[:,4]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_validation = le.fit_transform(Y_validation)

### Pomocne funkcie

In [ ]:
import graphviz
from sklearn import tree

In [ ]:
def viz_tree(clf, class_names, feature_names):
    tree_ = tree.export_graphviz(clf, out_file = None, filled=True, rounded=True, class_names=class_names, feature_names=feature_names)
    graph = graphviz.Source(tree_)
    return graph

### Natrenovanie rozhodovacieho stromu

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier(max_depth=3)

clf.fit(X_train, Y_train)

In [ ]:
pred = clf.predict(X_validation)

In [ ]:
print(accuracy_score(Y_validation, pred))
print(classification_report(Y_validation, pred))

In [ ]:
feature_names_array = ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm']

In [ ]:
viz_tree(clf, data['class'].unique(), feature_names_array)

In [ ]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold

In [ ]:
print(children_left)
print(children_right)

In [ ]:
print(feature)

-------
## Important classes for single tree interpretation

In [ ]:
def get_leaf_class_name(class_names, node_values):
    max_value_index = 0
    max_value = 0
    for c, value in enumerate(node_values[0]):
        if value != 0 and value > max_value:
            max_value_index = c
            max_value = value
    
    return class_names[max_value_index]

        
def get_node_value(node_values):
    max_ = 0
    for c, value in enumerate(node_values[0]):
        if value > max_:
            max_ = value
            
    return max_
        
        
def get_paths_for(class_name, tree, cur_index, lst, paths, sign):
    if cur_index >= len(tree.children_left):
        return
    
    if len(lst) > 0 and cur_index != -1:
        lst[len(lst) - 1] = (lst[len(lst) - 1][0], lst[len(lst) - 1][1], sign, lst[len(lst) - 1][3])
        
    lst.append((cur_index, tree.feature[cur_index], sign, round(tree.threshold[cur_index], 2)))
    if cur_index != -1:
        get_paths_for(class_name, tree, tree.children_left[cur_index], lst, paths, '<=')
        get_paths_for(class_name, tree, tree.children_right[cur_index], lst, paths, '>')
    else:
        if get_leaf_class_name(data['class'].unique(), tree.value[list(lst[-2])[0]]) == class_name:
            paths.add(tuple(lst[:-1]))
        
    lst.pop()
    
    
def get_feature_value_pairs(tree, paths):
    
    pairs = []
    for path in paths:
        d = []
        features = tree.feature
        thresholds = tree.threshold

        for i, p in enumerate(list(path[:-1])):
            d.append((p, features[p], thresholds[p]))

        pairs.append(d)
        
    return pairs


# One list item explanation
# [value_1, value_2, value_3]
# value_1 - ID of the node in the tree
# value_2 - numeric encoded feature value
# value_3 - threshold for each encoded node
def compose_rules(nodes, tree):
    rules = []
    
    for node in nodes:
        
        max_n_count = 0
        rule = None
        for i, val in enumerate(node):
            
            s = val[2]
            c = 1
            
            # This sets the first rule to be as the decision rule, because no matches were found on that level
            if rule == None:
                rule = [feature_names_array[val[1]], val[2]]
            
            for val2 in node[(i + 1):]:
                # Count same rules on that level
                if val[1] == val2[1]:
                    s += val2[2]
                    c += 1
                else:
                    break
            
            if c > max_n_count:
                max_n_count = c
                rule = [feature_names_array[val[1]], s / c]
        
        
        if rule != None:
            rules.append(rule)
        
    return rules


def get_global_paths_for(class_name, rf_model):
    level_features = [[] for x in range(5)]
    
    for tree in rf_model.estimators_:
        lst = []
        paths = set()
        
        get_paths_for(class_name, tree.tree_, 0, lst, paths)
        outputs = get_feature_value_pairs(tree.tree_, list(paths))
        
        for arr in outputs:
            for level, value in enumerate(arr):
                level_features[level].append(list(value))
        
    return level_features

In [ ]:
def rule_predict(class_name, rule):
    pred = []
    
    for row in X_validation:
        
        valid = True
        for condition in rule[:-1]:
            condition = list(condition)
            
            if not get_truth(row[condition[1]], condition[2], condition[3]):
                valid = False
            
        if valid:
            pred.append(class_name)
        else:
            pred.append(-1)
            
    return pred


def compute_rule_accuracy(class_name, rule):
    pred = rule_predict(class_name, rule)
    report = classification_report(pred, Y_validation, output_dict = True)
    
    return report[str(class_name)]['precision']


def prune_rule(class_name, rule):
    max_accuracy = 0
    final_rule = rule
    
    for i in range(len(rule[:-1])):
        accuracy = compute_rule_accuracy(class_name, rule[i:])
        
        if accuracy > max_accuracy:
            final_rule = rule[i:]
            max_accuracy = accuracy
           
    return (max_accuracy, tuple(final_rule))


def combine_rules(class_name, rf_model):
    rules = set()
    
    for estimator in rf_model.estimators_:
        get_paths_for(class_name, estimator.tree_, 0, [], rules, '<=')
        
    return rules


def get_class_index(class_name):
    for c, cls in enumerate(data['class'].unique()):
        if cls == class_name:
            return c
        
        
def get_rules_accuracy(class_name, rf_model):
    rules = list(combine_rules(class_name, rf_model))
    
    final_rules = set()
    for rule in rules:
        rule_ = [compute_rule_accuracy(get_class_index(class_name), rule), rule]
        final_rules.add(tuple(rule_))
        
    return sorted(list(final_rules), reverse=True, key=lambda x: list(x)[0])
    
        
def get_pruned_rules_accuracy(class_name, rf_model):
    rules = list(combine_rules(class_name, rf_model))
    
    final_rules = set()
    for rule in rules:
        pruned = prune_rule(get_class_index(class_name), list(rule))
        final_rules.add(tuple(pruned))
    
    return sorted(list(final_rules), reverse=True, key=lambda x: list(x)[0])

In [ ]:
rules = set()
get_paths_for("Iris-virginica", clf.tree_, 0, [], rules, '<=')
rules

In [ ]:
data['class'].unique()

In [ ]:
compute_rule_accuracy(2, [(0, 2, '>', 2.45), (2, 3, '<=', 1.7), (3, 2, '>', 5.0), (5, -2, '>', -2.0)])

In [ ]:
prune_rule(2, [(0, 2, '>', 2.45), (2, 3, '<=', 1.7), (3, 2, '>', 5.0), (5, -2, '>', -2.0)])

In [ ]:
compute_rule_accuracy(2, [(0, 2, '>', 2.45), (2, 3, '>', 1.7), (6, -2, '>', -2.0)])

In [ ]:
prune_rule(2, [(0, 2, '>', 2.45), (2, 3, '>', 1.7), (6, -2, '>', -2.0)])

In [ ]:
p

-----
## Natrenovanie random forestu

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfclf = RandomForestClassifier(n_estimators=5, max_depth=4)
rfclf.fit(X_train, Y_train)

In [ ]:
pred = rfclf.predict(X_validation)

In [ ]:
print(accuracy_score(Y_validation, pred))
print(classification_report(Y_validation, pred))

In [ ]:
viz_tree(rfclf.estimators_[0], data['class'].unique(), feature_names_array)

In [ ]:
p = get_pruned_rules_accuracy("Iris-versicolor", rfclf)

In [ ]:
p

In [ ]:
p2 = get_rules_accuracy("Iris-versicolor", rfclf)

In [ ]:
p2

In [ ]:
rfclf.estimators_[0].tree_.feature

In [ ]:
data['class'].unique()

In [ ]:
lst = []
t1_paths = set()

get_paths_for("Iris-versicolor", rfclf.estimators_[0].tree_, 0, lst, t1_paths, '<=')
t1_paths

In [ ]:
viz_tree(rfclf.estimators_[1], data['class'].unique(), feature_names_array)

In [ ]:
rfclf.estimators_[1].tree_.feature

In [ ]:
lst = []
t2_paths = set()

get_paths_for("Iris-versicolor", rfclf.estimators_[1].tree_, 0, lst, t2_paths, '<=')
t2_paths

In [ ]:
viz_tree(rfclf.estimators_[2], data['class'].unique(), feature_names_array)

In [ ]:
rfclf.estimators_[2].tree_.feature

In [ ]:
lst = []
t3_paths = set()

get_paths_for("Iris-versicolor", rfclf.estimators_[2].tree_, 0, lst, t3_paths, '<=')
t3_paths

In [ ]:
viz_tree(rfclf.estimators_[3], data['class'].unique(), feature_names_array)

In [ ]:
rfclf.estimators_[3].tree_.feature

In [ ]:
lst = []
t4_paths = set()

get_paths_for("Iris-versicolor", rfclf.estimators_[3].tree_, 0, lst, t4_paths, '<=')
t4_paths

In [ ]:
viz_tree(rfclf.estimators_[4], data['class'].unique(), feature_names_array)

In [ ]:
lst = []
t5_paths = set()

get_paths_for("Iris-versicolor", rfclf.estimators_[4].tree_, 0, lst, t5_paths, '<=')
t5_paths

In [ ]:
[(1.0, ((1, 2, '>', 2.45), (3, -2, '>', -2.0))),
 (1.0, ((3, 2, '<=', 5.9), (4, -2, '<=', -2.0))),
 (1.0, ((5, 3, '<=', 1.8), (6, -2, '<=', -2.0))),
 (0.9166666666666666, ((5, 3, '<=', 1.75), (6, -2, '<=', -2.0))),
 (0.9166666666666666, ((3, 3, '<=', 1.7), (4, -2, '<=', -2.0))),
 (0.8333333333333334,
  ((6, 2, '<=', 4.95), (7, 3, '<=', 1.65), (8, -2, '<=', -2.0))),
 (0.8333333333333334, ((8, 3, '<=', 1.65), (9, -2, '<=', -2.0))),
 (0.8333333333333334,
  ((0, 3, '>', 0.8), (2, 2, '<=', 4.85), (3, -2, '<=', -2.0))),
 (0.75, ((0, 2, '>', 2.6), (2, 2, '<=', 4.75), (3, -2, '<=', -2.0))),
 (0.6666666666666666, ((5, 0, '<=', 6.05), (6, -2, '<=', -2.0)))]

In [ ]:
t1_paths

In [ ]:
t2_paths

In [ ]:
t3_paths

In [ ]:
t4_paths

In [ ]:
t5_paths

In [ ]:
feature_names_array

In [ ]:
versicolor = get_global_paths_for("Iris-versicolor", rfclf)
virginica = get_global_paths_for("Iris-virginica", rfclf)
setosa = get_global_paths_for("Iris-setosa", rfclf)

In [ ]:
setosa

In [ ]:
ver_rules = compose_rules(versicolor, rfclf.estimators_[0].tree_)
vir_rules = compose_rules(virginica, rfclf.estimators_[0].tree_)
set_rules = compose_rules(setosa, rfclf.estimators_[0].tree_)

In [ ]:
ver_rules

In [ ]:
vir_rules

In [ ]:
set_rules

In [ ]:
X = X_validation

In [ ]:
Y_validation

In [ ]:
X

In [ ]:
feature_names_array

In [ ]:
Y_validation

In [ ]:
data['class'].unique()

In [ ]:
def test():
    pred = []
    for el in X[:,2]:
        if el <= 2.5:
            pred.append(0)
        elif el > 2.5 and el <= 4.8:
            pred.append(1)
        else:
            pred.append(2)
            
    return pred

In [ ]:
test()

In [ ]:
print(accuracy_score(Y_validation, test()))

In [ ]:
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth

while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

        
print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % ("\t", i))
    else:
        print("%snode=%s test node: go to node %s if %s <= %s else to "
              "node %s."
              % ("\t",
                 i,
                 children_left[i],
                 feature_names_array[feature[i]],
                 threshold[i],
                 children_right[i],
                 ))
print()


In [ ]:
node_indicator = clf.decision_path(X_validation)

# Similarly, we can also have the leaves ids reached by each sample.

leave_id = clf.apply(X_validation)

# Now, it's possible to get the tests that were used to predict a sample or
# a group of samples. First, let's make it for the sample.

sample_id = 3
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:
    if leave_id[sample_id] == node_id:
        continue

    if (X_validation[sample_id, feature[node_id]] <= threshold[node_id]):
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print("decision id node %s : (X_test[%s, %s] (= %s) %s %s)"
          % (node_id,
             sample_id,
             feature[node_id],
             X_validation[sample_id, feature[node_id]],
             threshold_sign,
             threshold[node_id]))

# For a group of samples, we have the following common node.
sample_ids = [0, 1]
common_nodes = (node_indicator.toarray()[sample_ids].sum(axis=0) ==
                len(sample_ids))

common_node_id = np.arange(n_nodes)[common_nodes]

print("\nThe following samples %s share the node %s in the tree"
      % (sample_ids, common_node_id))
print("It is %s %% of all nodes." % (100 * len(common_node_id) / n_nodes,))

In [ ]:
node_index

In [ ]:
clf.tree_.value

In [ ]:
def traverse(target_class, tree, cur_level):
    if get_leaf_class_name(data['class'].unique(), tree.value) == target_class:
        return